In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/kellykindla/Desktop/Module6/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,76.53,67,96,13.00,overcast clouds
1,1,Esso,RU,55.9333,158.7000,9.39,90,100,2.48,overcast clouds
2,2,Mahibadhoo,MV,3.7833,72.9667,81.73,78,100,13.24,overcast clouds
3,3,Samarai,PG,-10.6167,150.6667,82.06,81,77,7.92,broken clouds
4,4,Barrow,US,71.2906,-156.7887,-16.94,98,100,7.52,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,76.53,67,96,13.00,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,77.70,80,25,21.52,light rain
6,6,Saldanha,ZA,-33.0117,17.9442,64.33,78,5,9.35,clear sky
8,8,Kapaa,US,22.0752,-159.3190,74.23,64,27,17.22,scattered clouds
9,9,Cape Town,ZA,-33.9258,18.4232,65.73,70,8,6.53,clear sky
16,16,Constitucion,CL,-35.3333,-72.4167,67.12,51,0,15.57,clear sky
22,22,Mar Del Plata,AR,-38.0023,-57.5575,65.12,73,80,22.24,broken clouds
26,26,Beloha,MG,-25.1667,45.0500,69.26,76,0,16.49,clear sky
29,29,Puerto Ayora,EC,-0.7393,-90.3518,71.69,79,81,10.25,broken clouds
31,31,Cabo San Lucas,MX,22.8909,-109.9124,79.05,61,78,2.19,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                253
City                   253
Country                253
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,76.53,overcast clouds,14.8961,-24.4956,
5,Atuona,PF,77.70,light rain,-9.8000,-139.0333,
6,Saldanha,ZA,64.33,clear sky,-33.0117,17.9442,
8,Kapaa,US,74.23,scattered clouds,22.0752,-159.3190,
9,Cape Town,ZA,65.73,clear sky,-33.9258,18.4232,
16,Constitucion,CL,67.12,clear sky,-35.3333,-72.4167,
22,Mar Del Plata,AR,65.12,broken clouds,-38.0023,-57.5575,
26,Beloha,MG,69.26,clear sky,-25.1667,45.0500,
29,Puerto Ayora,EC,71.69,broken clouds,-0.7393,-90.3518,
31,Cabo San Lucas,MX,79.05,broken clouds,22.8909,-109.9124,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame and add them to location key for params
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city (try->except).
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
#first replace blank hotel name strings with NaNs 
replace_hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
clean_hotel_df = replace_hotel_df.dropna()
clean_hotel_df.sample(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
371,Guaraniacu,BR,78.22,overcast clouds,-25.1008,-52.8781,Dalla´s Hotel
674,Baghmara,IN,64.74,overcast clouds,23.7950,86.2085,Shankar tea shop
585,Ambovombe,MG,69.58,clear sky,-25.1667,46.0833,RELAHARA
209,San Quintin,MX,70.90,clear sky,30.4833,-115.9500,Don Eddie's Sport Fishing Center
439,Gat,IL,74.59,broken clouds,31.6100,34.7642,OlusHome
148,Arraial Do Cabo,BR,76.39,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
47,Georgetown,MY,76.53,moderate rain,5.4112,100.3354,Cititel Penang
485,Puerto Penasco,MX,72.91,clear sky,31.3167,-113.5333,Playa Bonita Resort
34,Rikitea,PF,75.81,light rain,-23.1203,-134.9692,People ThankYou
277,Los Llanos De Aridane,ES,63.07,scattered clouds,28.6585,-17.9182,Valle Aridane


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,dissipating=False,
             max_intensity=300, point_radius=4)

#create markers for each hotel- include info_box_content for heach marker 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))